In [1]:
import os
import pandas as pd
import zipfile
import csv
from io import StringIO
import mysql.connector
import re

### Testing on home server

In [2]:
try:
    connection = mysql.connector.connect(
        host = '192.168.1.26',
        user = 'aportra',
        password = '@$Occer22',
        database = 'wedge_assignment')

    cursor = connection.cursor()
except mysql.connector.Error as err:
    print(f'{err}')

finally:
    if connection.is_connected():
        connection.close()
        print('Connection Closed')
    
    if cursor:
        print('Cursor created')

Connection Closed
Cursor created


#### Setting Directory for Mac and Windows 

In [3]:

try:
    directory = '/Users/aaronportra/Documents/Advanced Data Analytics/wedge_project/WedgeZipOfZips_small/'
    os.listdir(directory)
except FileNotFoundError:
    directory = 'C:\\Users\\aport\\OneDrive\\Documents\\School\\Fall Semester 2024\\Applied Data Analytics\\wedge_project\\WedgeZipOfZips_small\\'
    os.listdir(directory)

## Cleaning Data for BQ

In [4]:
def sniff(file_path):
    with open(file_path, 'r', errors = 'replace') as f:
        sample = f.read(10000)

        sniffer = csv.Sniffer()

        f.seek(0)

        delimiter = sniffer.sniff(sample).delimiter

        f.seek(0)

        reader = csv.reader(f, delimiter=delimiter)

        first_row = next(reader)

        has_header = sniffer.has_header(sample)


        if len(first_row) < 50:
            has_header = False

    return delimiter, has_header

In [5]:
sniff(directory + 'transArchive_201607_small.zip')

(',', False)

In [6]:
sniff(directory + 'transArchive_201201_201203_inactive_small.zip')

(';', True)

In [9]:
transactions = {}

for file in os.listdir(directory):
    if file.endswith('.zip'):
        with zipfile.ZipFile(directory + file,'r') as zip_file:
            for info in zip_file.namelist():
                h = sniff(directory + file)

                file_content = zip_file.read(info).decode('utf-8')

                csv_file = StringIO(file_content)

                if h[1]:
                    data = pd.read_csv(csv_file, delimiter = h[0])
                    data = pd.DataFrame(data)

                    columns = data.columns


                elif not h[1]:
                    data = pd.read_csv(csv_file, delimiter = h[0], header = None)
                    data = pd.DataFrame(data)
                    data.columns = columns
                
                transactions[file.rstrip('.zip')] = data

            
                

In [80]:
for trans in transactions:
   assert(len(transactions[trans].columns) == 50)

In [81]:
def del_nan(data):
    for trans in data:
        data[trans] = data[trans].fillna('')

def add_space(text):
    return re.sub(r'([a-z])([A-Z])',r'\1 \2',text)



In [11]:
transactions['transArchive_201607_small']

,datetime,register_no,emp_no,trans_no,upc,description,trans_type,trans_subtype,trans_status,department,...,batchHeaderID,local,organic,display,receipt,card_no,store,branch,match_id,trans_id
0,2016-07-01 07:06:15,51,94,1,0065722700050,Electrolyte Water 1.5L Essenti,I,,,1,...,\N,0,0,NaN,0,3,1,3,0,1
1,2016-07-01 08:39:21,51,94,38,DISCOUNT,Discount,I,NaN,NaN,0,...,\N,0,\N,NaN,0,3,1,3,0,9
2,2016-07-01 09:20:01,51,94,58,0,Credit Card,T,CC,NaN,0,...,\N,0,\N,NaN,0,3,1,3,0,5
3,2016-07-01 09:30:13,52,75,1,0085828400298,Salmon Salt&Pep.Bar 1.5oz Epic,I,,,1,...,\N,0,0,NaN,0,55887,1,3,0,7
4,2016-07-01 08:42:38,51,94,41,0000000000129,APPLES Pink Lady Organic,I,,,2,...,\N,0,1,NaN,0,17940,1,3,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,2016-07-01 12:26:17,3,97,50,TAX,Tax,A,NaN,NaN,0,...,\N,0,\N,NaN,0,22587,1,0,0,11
9997,2016-07-01 12:26:17,5,27,13,0,Credit Card,T,CC,NaN,0,...,\N,0,\N,NaN,0,11244,1,0,0,23
9998,2016-07-01 12:26:18,5,27,13,0,Change,T,CA,NaN,0,...,\N,0,\N,NaN,0,11244,1,0,0,24
9999,2016-07-01 12:26:18,5,27,13,DISCOUNT,Discount,I,NaN,NaN,0,...,\N,0,\N,NaN,0,11244,1,0,0,25


In [83]:
# retrieve_nans(transactions)